In [42]:
import pandas as pd 
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn

ModuleNotFoundError: No module named 'mlflow'

In [34]:
dataset = pd.read_excel("dataset_prix_appartements_paris.xlsx")
dataset.head()

,surface_m2,nombre_pieces,nombre_chambres,etage,ascenseur,balcon,parking,arrondissement,prix_m2_moyen_arrondissement,prix_euros
0,74,2,1,0,0,0,0,20,8000,521311
1,58,2,1,2,0,0,0,7,12800,756370
2,105,4,3,1,0,0,1,17,10800,1255174
3,35,1,1,4,1,1,1,11,10400,427580
4,51,2,1,8,1,1,0,17,10800,654948


In [35]:
num_features = ["surface_m2", "etage", "prix_m2_moyen_arrondissement"]
cat_features = ["arrondissement"]
bin_features = ["ascenseur", "balcon", "parking"]

In [36]:
preprocessor = ColumnTransformer(
    transformers = [
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(
            drop = "first",
            handle_unknown = "ignore",
            sparse_output = False,
            dtype = int
        ), cat_features),
        ("bin", "passthrough", bin_features)
    ]
)

In [37]:
rf = RandomForestRegressor(
    n_estimators=100,   # nombre d'arbres
    max_depth=None,     # profondeur maximale, None = pas de limite
    random_state=42,
    n_jobs=-1           # utilise tous les cœurs CPU
)

In [38]:
model = Pipeline(
    steps = [
        ("preprocessing", preprocessor),
        ("forest",rf )
    ]
)

In [39]:
X = dataset.drop(columns="prix_euros")
y = dataset["prix_euros"]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['surface_m2', 'etage',
                                                   'prix_m2_moyen_arrondissement']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                dtype=<class 'int'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['arrondissement']),
                                                 ('bin', 'passthrough',
                                                  ['ascenseur', 'balcon',
                                                   'parking'])])),
                ('forest', RandomForestRegressor(n_jobs=-1, random_state=42))])

In [40]:
y_pred = model.predict(X_test)

In [41]:
r2 = r2_score(y_test, y_pred)
r2

0.9656323994566789